In [2]:
import warnings
warnings.filterwarnings("ignore")

In [26]:
from keras.datasets import mnist

from keras.layers import *
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
import keras
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam

In [7]:
(x_train,_),(_,_)=mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [8]:
x_train.shape

(60000, 28, 28)

In [9]:
x_train=(x_train-127.5)/127.5

print(x_train.min())
print(x_train.max())

-1.0
1.0


In [15]:
TOTAL_EPOCHS=50
BATCH_SIZE=256
HALF_BATCH=128

NO_OF_BATCHES=int (x_train.shape[0]/BATCH_SIZE)

NOISE_DIM=100
adam= Adam(lr=2e-4,beta_1=0.5)

In [19]:
#Generator Model : Upsampling

generator=Sequential()
generator.add(Dense(units=7*7*128,input_shape=(NOISE_DIM,)))
generator.add(Reshape((7,7,128)))
generator.add(LeakyReLU(0.2))
generator.add(BatchNormalization())

#(7,7,128) -> (14,14,64)

generator.add(Conv2DTranspose(64,(3,3),strides=(2,2),padding='same'))
generator.add(LeakyReLU(0.2))
generator.add(BatchNormalization())

#(14,14,64) -> (28,28,1)

generator.add(Conv2DTranspose(1,(3,3), strides=(2,2), padding='same',activation='tanh'))

generator.compile(loss=keras.losses.binary_crossentropy, optimizer=adam)

generator.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 6272)              633472    
                                                                 
 reshape_1 (Reshape)         (None, 7, 7, 128)         0         
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 7, 7, 128)         0         
                                                                 
 batch_normalization_1 (Batc  (None, 7, 7, 128)        512       
 hNormalization)                                                 
                                                                 
 conv2d_transpose_1 (Conv2DT  (None, 14, 14, 64)       73792     
 ranspose)                                                       
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 14, 14, 64)       

In [21]:
#Discriminator Modelb -> i.e Down Sampling

#(28,28,1) -> (14,14,64)

discriminator=Sequential()
discriminator.add(Conv2D(64,kernel_size=(3,3),strides=(2,2),padding='same',input_shape=(28,28,1)))
discriminator.add(LeakyReLU(0.2))

#(14,14,64) -> (7,7,128)
discriminator.add(Conv2D(128,kernel_size=(3,3),strides=(2,2),padding='same'))
discriminator.add(LeakyReLU(0.2))

#(7,7,128) -> 6272
discriminator.add(Flatten())
discriminator.add(Dense(100))
discriminator.add(LeakyReLU(0.2))

discriminator.add(Dense(1, activation='sigmoid'))

discriminator.compile(loss=keras.losses.binary_crossentropy, optimizer=adam)

discriminator.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 14, 14, 64)        640       
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 14, 14, 64)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 7, 7, 128)         73856     
                                                                 
 leaky_re_lu_7 (LeakyReLU)   (None, 7, 7, 128)         0         
                                                                 
 flatten_1 (Flatten)         (None, 6272)              0         
                                                                 
 dense_5 (Dense)             (None, 100)               627300    
                                                                 
 leaky_re_lu_8 (LeakyReLU)   (None, 100)              

In [22]:
# Combined Model

discriminator.trainable=False

gan_input= Input(shape=(NOISE_DIM, ))

generated_img= generator(gan_input)

gan_output=discriminator(generated_img)

# Functional API

model=Model(gan_input,gan_output)

model.compile(loss=keras.losses.binary_crossentropy, optimizer= adam)

In [23]:
#model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 sequential_2 (Sequential)   (None, 28, 28, 1)         708609    
                                                                 
 sequential_4 (Sequential)   (None, 1)                 701897    
                                                                 
Total params: 1,410,506
Trainable params: 708,225
Non-trainable params: 702,281
_________________________________________________________________


In [24]:
x_train=x_train.reshape(-1,28,28,1)

In [25]:
x_train.shape

(60000, 28, 28, 1)

In [ ]:
# Displaying images

def display_images(samples= 25):

  noise=np.random.normal(0,1,size(samples, NOISE_DIM))

  generated_img=generator.predict(noise)

  plt.figure(figsize=(10,10))

  for i in range(samples):
    plt.subplot(5,5,i+1)
    plt.imgshow(generated_img[i].reshape(28,28), cmap='binary')
    plt.axis('off')

  plt.show()

In [ ]:
# Training Loop

d_losses=[]
g_losses=[]

for epoch in range(TOTAL_EPOCHS):
  epoch_d_loss=0.0
  epoch_g_loss=0.0

  # Mini batch gradient decent
  for step in range(NO_OF_BATCHES):
    # Step 1 train descriminate
    discriminator.trainable = True

    # get the real data
    idx= np.random.randit(0,60000, HALF_BATCH)
    real_imgs=x_train[idx]

    # get the fake data
    noise=np.random.normal(0,1,size=(HALF_BATCH,NOISE_DIM))
    fake_imgs=generator.predict(noise)

    # Labels
    real_y=np.ones((HALF_BATCH,1))*0.9
    fake_y=np.zeros((HALF_BATCH,1))

    # Train D
    d_loss_real = discriminator.train_on_batch(real_imgs,real_y)
    d_loss_fake = discriminator.train_on_batch(fake_imgs,fake_y)

    d_loss=0.5*d_loss_real +0.5*d_loss_fake

    epoch_d_loss +=d_losses

    # Step 2 train generate

    discriminator.trainable = False

    noise=np.random.normal(0,1,size=(BATCH_SIZE,NOISE_DIM))

    ground_truth_y= np.ones((BATCH_SIZE,1))

    epoch_g_loss += g_losses

    # Display
    print(f"Epoch{epoch+1}, Disc loss {epoch_d_loss/NO_OF_BATCHES}, Generator loss {epoch_g_loss/NO_OF_BATCHES}")

    d_losses.append(epoch_d_loss/NO_OF_BATCHES)
    g_losses.append(epoch_g_loss/NO_OF_BATCHES)

    if (epoch+1)% 10 ==0:
      generator.save("generator.h5")
      display_images()
